In [8]:
import casadi as cs
import urdf2casadi.urdf2casadi.converter as converter
from urdf_parser_py.urdf import URDF, Pose
from urdf2casadi.casadi_geom import dual_quaternion_to_transformation_matrix
import os # For current directory

ImportError: No module named urdf2casadi.converter

In [2]:
# urdf2casadi uses cs.MX and cs.SX, which can be hard to read as these are sparse matrices.
# This short function just makes it so that the result will be a numpy matrix
# Use for 
def cs2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()
# NOTE: casadi imports numpy as np, so cs.np is numpy

# Importing thrivaldi URDF

In [6]:
urdf_path = "./urdf/thrivaldi.urdf"
root_link = "gantry_base_link"
end_link = "gantry_tool0"
fk_dict = converter.from_file(root_link, end_link, urdf_path)
print fk_dict.keys()
converter.from_file?

NameError: name 'converter' is not defined

In [5]:
self.robot_desc = URDF.from_xml_file(urdf_path)
chain = robot_desc.get_chain(root, tip)
print(chain)

SyntaxError: invalid syntax (<ipython-input-5-c9d788b634b3>, line 1)

In [4]:
# CasADi MX symbol giving the joint symbols:
q = fk_dict["q"]
# Upper limits of the joint values
q_upper = fk_dict["upper"]
# Lower limits of the joint values
q_lower = fk_dict["lower"]
# Joint names
joint_names = fk_dict["joint_names"]
print "Number of joints:", q.size()[0]
print "Upper limits:", q_upper
print "Lower limits:", q_lower
print "Joint names:", joint_names

Number of joints: 6
Upper limits: [3.22885911619, 0.610865238198, 2.68780704807, 6.10865238198, 2.26892802759, 6.10865238198]
Lower limits: [-3.22885911619, -2.70526034059, -2.26892802759, -6.10865238198, -2.26892802759, -6.10865238198]
Joint names: ['gantry_joint_a1', 'gantry_joint_a2', 'gantry_joint_a3', 'gantry_joint_a4', 'gantry_joint_a5', 'gantry_joint_a6']


# Forward Kinematics

In [5]:
# CasADi MX symbol for transformation matrix of the forward kinematics:
T_fk_sym = fk_dict["T_fk"]
# CasADi MX symbol for dual_quaternion of the forward kinematics:
Q_fk_sym = fk_dict["dual_quaternion_fk"]

In [13]:
# Transformation matrix function:
T_fk = cs.Function("T_fk",[q], [T_fk_sym], ["nasdnasnd"], ["T_fk"])
# Dual quaternion function:
Q_fk = cs.Function("Q_fk",[q], [Q_fk_sym], ["q"], ["Q_fk"])
T_fk

Function(T_fk:(i0[6])->(o0[4x4]) MXFunction)

In [7]:
T0 = T_fk([0., 0., 0., 0., 0., 0.])
p0 = T0[:3, 3]
R0 = T0[:3, :3]
print "Transformation matrix:\n",cs2np(T0)
print "Position:\n", "x:",p0[0]," y:", p0[1], " z:", p0[2]
print "Distance from origin:\n", cs.np.linalg.norm(p0), "m"

Transformation matrix:
[[ 4.89658886e-12  0.00000000e+00  1.00000000e+00  1.76800000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  4.89658886e-12  6.40000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Position:
x: 1.768  y: 0  z: 0.64
Distance from origin:
1.8802723207025094 m



# Jacobians


In [15]:
# Create symbols
fk_position_jacobian_sym = cs.jacobian(T_fk_sym[:3,3], q)
fk_rotation_jacobian_sym = cs.jacobian(T_fk_sym[:3,:3], q)
fk_dual_quaternion_jacobian_sym = cs.jacobian(Q_fk_sym, q)

In [16]:
# Create functions
fk_position_jacobian = cs.Function("jac_fk_pos", [q], [fk_position_jacobian_sym], ["q"], ["jac_fk_pos"])
fk_rotation_jacobian = cs.Function("jac_fk_rot", [q], [fk_rotation_jacobian_sym], ["q"], ["jac_fk_rot"])
fk_dual_quaternion_jacobian = cs.Function("jac_fk_Q", [q], [fk_dual_quaternion_jacobian_sym], ["q"], ["jac_fk_Q"])

In [17]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
pos_jac = fk_position_jacobian(joint_vals)
print "Positional jacobian at ", joint_vals, "is:\n", cs2np(pos_jac)

Positional jacobian at  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0] is:
[[ 0.    -0.035 -0.035  0.     0.     0.   ]
 [-1.768  0.     0.     0.     0.     0.   ]
 [ 0.    -1.508 -0.828  0.    -0.158  0.   ]]


In [18]:
print "Comparative measure of how readily each direction is controlled:"
print "Norm of jac in x direction:", cs.np.linalg.norm(cs2np(pos_jac[0,:]))
print "Norm of jac in y direction:", cs.np.linalg.norm(cs2np(pos_jac[1,:]))
print "Norm of jac in z direction:", cs.np.linalg.norm(cs2np(pos_jac[2,:]))
print "\nComparative measure of how readily each joint affects positions:"
for i in xrange(len(joint_names)):
    print joint_names[i]+":",cs.np.linalg.norm(cs2np(pos_jac[:,i]))

Comparative measure of how readily each direction is controlled:
Norm of jac in x direction: 0.04949747468305833
Norm of jac in y direction: 1.768
Norm of jac in z direction: 1.727602963646451

Comparative measure of how readily each joint affects positions:
gantry_joint_a1: 1.768
gantry_joint_a2: 1.5084061124246348
gantry_joint_a3: 0.8287394041554921
gantry_joint_a4: 0.0
gantry_joint_a5: 0.158
gantry_joint_a6: 0.0


In [19]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
rot_jac = fk_rotation_jacobian(joint_vals)
print cs2np(rot_jac)

[[ 0.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  -1.00000000e+00  0.00000000e+00]
 [-4.89658886e-12  0.00000000e+00  0.00000000e+00 -1.00000000e+00
   0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00 -4.89658886e-12 -4.89658886e-12  0.00000000e+00
  -4.89658886e-12  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+00
   0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  4.89658886e-12  4.89658886e-12  0.00000000e+00
   4.89658886e-12  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  0.00000000e+00  4.89658886e-12
   0.00000000e+00  4.89658886e-12]
 [ 0.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  -1.00000000e+00  0.00000000e+00]]


In [21]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
Q_jac = fk_dual_quaternion_jacobian(joint_vals)
print cs2np(Q_jac)
print cs2np(Q_jac)[:4,0]

 [[ 0.35355339  0.          0.         -0.35355339  0.         -0.35355339]
 [ 0.          0.35355339  0.35355339  0.          0.35355339  0.        ]
 [-0.35355339  0.          0.         -0.35355339  0.         -0.35355339]
 [ 0.         -0.35355339 -0.35355339  0.         -0.35355339  0.        ]
 [ 0.          0.09510586 -0.14531044  0.         -0.36981685  0.        ]
 [-0.19940411  0.          0.          0.19940411  0.          0.19940411]
 [ 0.         -0.34612877 -0.10571246  0.          0.14354268  0.        ]
 [ 0.42567828  0.          0.          0.42567828  0.          0.42567828]]
[ 0.35355339  0.         -0.35355339  0.        ]


In [22]:
print "Comparative measure of joint effect on rotation"
for i in xrange(len(joint_names)):
    print joint_names[i]+":",cs.np.linalg.norm(cs2np(Q_jac)[:4,i]) # First four elements = rotation quaternion

Comparative measure of joint effect on rotation
gantry_joint_a1: 0.5
gantry_joint_a2: 0.5
gantry_joint_a3: 0.5
gantry_joint_a4: 0.5
gantry_joint_a5: 0.5
gantry_joint_a6: 0.5
